In [7]:
from datasets import load_dataset
import pandas as pd
import numpy as  np
import re

# Load the dataset and select the first 100,000 rows, then take a subset of 1,000
ds = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")
data = ds['train'].select(range(500000)).to_pandas()
# data_subset = data.head(1000)  # Subset for demonstration
data_subset = data

####################################
# Drop rows with NaN values in 'input' or 'output'
data_subset.dropna(subset=['input', 'output'], inplace=True)

# Drop rows with empty strings or only one word in either 'input' or 'output'
def is_valid(text):
    return isinstance(text, str) and len(text.strip().split()) > 1

data_subset = data_subset[data_subset['input'].apply(is_valid)]
data_subset = data_subset[data_subset['output'].apply(is_valid)]

# Reset index after filtering
data_subset.reset_index(drop=True, inplace=True)

print("Cleaned subset shape:", data_subset.shape)
############################################################


# Preprocess text: lowercase and remove special characters
def preprocess_text(text):
    if not isinstance(text, str):  # Handle non-string inputs
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Apply preprocessing to input and output columns
data_subset['input'] = data_subset['input'].apply(preprocess_text)
data_subset['output'] = data_subset['output'].apply(preprocess_text)

print("Subset shape:", data_subset.shape)

/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/pandas/_libs/tslibs/__init__.py:40: RuntimeWarning: coroutine 'Translator.detect' was never awaited
  from pandas._libs.tslibs.conversion import localize_pydatetime
/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cleaned subset shape: (483348, 2)
Subset shape: (483348, 2)


In [5]:
pip install deep-translator


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from deep_translator import GoogleTranslator

def translate_spanish_to_english(text):
    try:
        # Try to translate assuming it's Spanish; if it's already English, the output will match the input
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        return translated
    except Exception as e:
        return f"Translation error: {e}"

# Example usage
spanish_text = "Hola, ¿cómo estás?"
english_text = translate_spanish_to_english(spanish_text)
print(f"Original text: {spanish_text}")
print(f"Translated text: {english_text}")

english_text_2 = "Hello, how are you?"
translated_text_2 = translate_spanish_to_english(english_text_2)
print(f"Original text: {english_text_2}")
print(f"Translated text: {translated_text_2}")


/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Original text: Hola, ¿cómo estás?
Translated text: Hello how are you?
Original text: Hello, how are you?
Translated text: Hello, how are you?


In [8]:
data_subset['conversation'] = data['input'] + data['output']

In [11]:
pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 37.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=9879c63eba16363bb5cd2b531c18cee92834262e0911a8f83cf59aa76ebee327
  Stored in directory: /Users/ritikojha/Library/Caches/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensures consistent results
DetectorFactory.seed = 0

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/re.py:275: RuntimeWarning: coroutine 'Translator.detect' was never awaited
  return pattern.translate(_special_chars_map)


ModuleNotFoundError: No module named 'langdetect'

In [ ]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False  # Return False if text is too short or can't be detected

# Apply the function and filter the DataFrame
df_filtered = df[df['text'].apply(is_english)].reset_index(drop=True)

print(df_filtered)

In [9]:
# Step 3: Load GloVe embeddings and compute sentence embeddings
import os
import urllib.request
import zipfile
from sklearn.feature_extraction.text import CountVectorizer

# Download GloVe if not already present
glove_path = "glove.6B.100d.txt"
if not os.path.exists(glove_path):
    url = "http://nlp.stanford.edu/data/glove.6B.zip"
    urllib.request.urlretrieve(url, "glove.6B.zip")
    with zipfile.ZipFile("glove.6B.zip", 'r') as zip_ref:
        zip_ref.extract("glove.6B.100d.txt")

# Load GloVe vectors into a dictionary
glove_embeddings = {}
with open(glove_path, 'r', encoding='utf8') as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

# Helper function to average GloVe vectors for a sentence
def get_sentence_embedding(text, embeddings, dim=100):
    words = text.split()
    vectors = [embeddings[word] for word in words if word in embeddings]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(dim)

# Compute embeddings for the mock data
data_subset['embedding'] = data_subset['conversation'].apply(lambda x: get_sentence_embedding(x, glove_embeddings))

# Stack embeddings into a matrix
embedding_matrix = np.vstack(data_subset['embedding'].values)

embedding_matrix.shape


(483348, 100)

In [17]:
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Assume embeddings_matrix is your array of word embeddings
embeddings_matrix = normalize(embedding_matrix, norm='l2')

In [18]:
import umap

# Reduce to 50 dimensions with UMAP
umap_reducer = umap.UMAP(n_components=50, random_state=42, n_neighbors=15, min_dist=0.1)
reduced_embeddings_umap = umap_reducer.fit_transform(embeddings_matrix)

print("UMAP reduced shape:", reduced_embeddings_umap.shape)

/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP reduced shape: (96391, 50)


In [19]:
from sklearn.decomposition import TruncatedSVD

# Reduce to 50 dimensions with SVD
svd = TruncatedSVD(n_components=50, random_state=42)
reduced_embeddings_svd = svd.fit_transform(embeddings_matrix)

print("SVD reduced shape:", reduced_embeddings_svd.shape)

SVD reduced shape: (96391, 50)


In [20]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=42, perplexity=5, init='pca')
reduced_embeddings_tsne = tsne.fit_transform(embeddings_matrix)


In [21]:
from sklearn.cluster import MiniBatchKMeans

# On UMAP-reduced data
minibatch_kmeans_umap = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=100)
clusters_minibatch_umap = minibatch_kmeans_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
minibatch_kmeans_svd = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=100)
clusters_minibatch_svd = minibatch_kmeans_svd.fit_predict(reduced_embeddings_svd)

minibatch_kmeans_tsne = MiniBatchKMeans(n_clusters=4, random_state=42, batch_size=100)
clusters_minibatch_tsne = minibatch_kmeans_tsne.fit_predict(reduced_embeddings_tsne)

In [22]:
from sklearn.cluster import KMeans

# On UMAP-reduced data
kmeans_umap = KMeans(n_clusters=4, random_state=42)
clusters_kmeans_umap = kmeans_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
kmeans_svd = KMeans(n_clusters=4, random_state=42)
clusters_kmeans_svd = kmeans_svd.fit_predict(reduced_embeddings_svd)

# On tsne-reduced data
kmeans_tsne = KMeans(n_clusters=4, random_state=42)
clusters_kmeans_tsne = kmeans_tsne.fit_predict(reduced_embeddings_tsne)



: 

In [23]:
from sklearn.cluster import DBSCAN

# On UMAP-reduced data
dbscan_umap = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_umap = dbscan_umap.fit_predict(reduced_embeddings_umap)

# On SVD-reduced data
dbscan_svd = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_svd = dbscan_svd.fit_predict(reduced_embeddings_svd)

dbscan_tsne = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan_tsne = dbscan_tsne.fit_predict(reduced_embeddings_tsne)

In [4]:
from sklearn.metrics import silhouette_score

# Function to safely compute silhouette score
def compute_silhouette(data, labels):
    unique_labels = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise for DBSCAN
    if unique_labels > 1:  # Need at least 2 clusters for silhouette score
        return silhouette_score(data, labels)
    return None

# Dictionary of methods and their data/labels
methods = {
    'Mini-Batch K-Means (UMAP)': (reduced_embeddings_umap, clusters_minibatch_umap),
    'Mini-Batch K-Means (SVD)': (reduced_embeddings_svd, clusters_minibatch_svd),
    'Mini-Batch K-Means (tsne)': (reduced_embeddings_tsne, clusters_minibatch_tsne),
    # 'K-Means (UMAP)': (reduced_embeddings_umap, clusters_kmeans_umap),
    # 'K-Means (SVD)': (reduced_embeddings_svd, clusters_kmeans_svd),
    # 'K-Means (tsne)': (reduced_embeddings_tsne, clusters_kmeans_tsne),
    'DBSCAN (UMAP)': (reduced_embeddings_umap, clusters_dbscan_umap),
    'DBSCAN (SVD)': (reduced_embeddings_svd, clusters_dbscan_svd),
    'DBSCAN (tsne)': (reduced_embeddings_tsne, clusters_dbscan_tsne),
    # 'Spectral Clustering (UMAP)': (reduced_embeddings_umap, clusters_spectral_umap),
    # 'Spectral Clustering (SVD)': (reduced_embeddings_svd, clusters_spectral_svd)
}

# Compute and print silhouette scores
for method, (data, labels) in methods.items():
    score = compute_silhouette(data, labels)
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if score is not None:
        print(f"{method}: Silhouette Score = {score:.4f}, Clusters = {num_clusters}")
    else:
        print(f"{method}: Cannot compute silhouette score (Clusters = {num_clusters})")


# cluster 4 results--
# Mini-Batch K-Means (UMAP): Silhouette Score = 0.3405, Clusters = 4
# Mini-Batch K-Means (SVD): Silhouette Score = 0.0552, Clusters = 4
# K-Means (UMAP): Silhouette Score = 0.3405, Clusters = 4
# K-Means (SVD): Silhouette Score = 0.0706, Clusters = 4
# DBSCAN (UMAP): Silhouette Score = -0.3862, Clusters = 19
# DBSCAN (SVD): Silhouette Score = -0.1877, Clusters = 29

NameError: name 'reduced_embeddings_svd' is not defined

In [6]:
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import umap
from sklearn.cluster import MiniBatchKMeans, DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score

# Step 4: Simulate GloVe-like embeddings (100d vectors)
np.random.seed(42)
mock_embeddings = np.random.rand(len(data_subset), 100)

# Step 5: Dimensionality reduction
# UMAP to 3D
umap_3d = umap.UMAP(n_components=3, random_state=42)
umap_result = umap_3d.fit_transform(mock_embeddings)

# LSA (SVD) to 3D
lsa = TruncatedSVD(n_components=3, random_state=42)
lsa_result = lsa.fit_transform(mock_embeddings)

# t-SNE to 3D
tsne = TSNE(n_components=3, random_state=42, perplexity=5, init='pca')
tsne_result = tsne.fit_transform(mock_embeddings)

# Step 6: Clustering (on UMAP-reduced data for consistency)
results = {}

# MiniBatch KMeans
kmeans = MiniBatchKMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans.fit_predict(umap_result)
results['kmeans'] = {
    'labels': kmeans_labels,
    'silhouette': silhouette_score(umap_result, kmeans_labels)
}

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2)
dbscan_labels = dbscan.fit_predict(umap_result)
# Filter out noise points (-1) before calculating silhouette
if len(set(dbscan_labels)) > 1 and -1 not in dbscan_labels:
    dbscan_silhouette = silhouette_score(umap_result, dbscan_labels)
else:
    dbscan_silhouette = None
results['dbscan'] = {
    'labels': dbscan_labels,
    'silhouette': dbscan_silhouette
}

# Spectral Clustering
# spectral = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=42)
# spectral_labels = spectral.fit_predict(umap_result)
# results['spectral'] = {
#     'labels': spectral_labels,
#     'silhouette': silhouette_score(umap_result, spectral_labels)
# }

results


/Users/ritikojha/Library/Python/3.9/lib/python/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


{'kmeans': {'labels': array([0, 0, 1, ..., 1, 0, 2], dtype=int32),
  'silhouette': 0.2340988},
 'dbscan': {'labels': array([0, 0, 0, ..., 0, 0, 0]), 'silhouette': None}}

In [8]:
# Re-simulate GloVe-like sentence embeddings (100d vectors)
np.random.seed(42)
mock_embeddings = np.random.rand(len(data_subset), 100)

# LSA (SVD) to 3D
lsa = TruncatedSVD(n_components=3, random_state=42)
lsa_result = lsa.fit_transform(mock_embeddings)

# t-SNE to 3D
tsne = TSNE(n_components=3, random_state=42, perplexity=5, init='pca')
tsne_result = tsne.fit_transform(mock_embeddings)

# Clustering on LSA-reduced data
results_revised = {}

# MiniBatch KMeans
kmeans_lsa = MiniBatchKMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans_lsa.fit_predict(lsa_result)
results_revised['kmeans_lsa'] = {
    'labels': kmeans_labels,
    'silhouette': silhouette_score(lsa_result, kmeans_labels)
}

# DBSCAN
dbscan_lsa = DBSCAN(eps=0.5, min_samples=2)
dbscan_labels = dbscan_lsa.fit_predict(lsa_result)
if len(set(dbscan_labels)) > 1 and -1 not in dbscan_labels:
    dbscan_silhouette = silhouette_score(lsa_result, dbscan_labels)
else:
    dbscan_silhouette = None
results_revised['dbscan_lsa'] = {
    'labels': dbscan_labels,
    'silhouette': dbscan_silhouette
}

# Spectral Clustering
# spectral_lsa = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=42)
# spectral_labels = spectral_lsa.fit_predict(lsa_result)
# results_revised['spectral_lsa'] = {
#     'labels': spectral_labels,
#     'silhouette': silhouette_score(lsa_result, spectral_labels)
# }

results_revised


{'kmeans_lsa': {'labels': array([2, 2, 2, ..., 0, 1, 2], dtype=int32),
  'silhouette': 0.2100656583744637},
 'dbscan_lsa': {'labels': array([0, 0, 0, ..., 0, 0, 0]), 'silhouette': None}}

In [12]:
import pandas as pd

# Collect clustering performance results
evaluation_summary = pd.DataFrame({
    # "Clustering Method": ["MiniBatch KMeans", "DBSCAN", "Spectral Clustering"],
    "Clustering Method": ["MiniBatch KMeans", "DBSCAN"],
    "Silhouette Score (LSA)": [
        results_revised['kmeans_lsa']['silhouette'],
        results_revised['dbscan_lsa']['silhouette'],
        # results_revised['spectral_lsa']['silhouette']
    ]
})


evaluation_summary


,Clustering Method,Silhouette Score (LSA)
0,MiniBatch KMeans,0.210066
1,DBSCAN,NaN


In [14]:
# Redefine results_revised (in case lost due to interruption)
from sklearn.cluster import MiniBatchKMeans, DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score

# Redefine mock_embeddings
np.random.seed(42)
mock_embeddings = np.random.rand(len(data_subset), 100)

# Dimensionality reduction using SVD (LSA)
lsa = TruncatedSVD(n_components=3, random_state=42)
lsa_result = lsa.fit_transform(mock_embeddings)

# Clustering results storage
results_revised = {}

# MiniBatch KMeans
kmeans_lsa = MiniBatchKMeans(n_clusters=3, random_state=42)
kmeans_labels = kmeans_lsa.fit_predict(lsa_result)
results_revised['kmeans_lsa'] = {
    'labels': kmeans_labels,
    'silhouette': silhouette_score(lsa_result, kmeans_labels)
}

# DBSCAN
dbscan_lsa = DBSCAN(eps=0.5, min_samples=2)
dbscan_labels = dbscan_lsa.fit_predict(lsa_result)
if len(set(dbscan_labels)) > 1 and -1 not in dbscan_labels:
    dbscan_silhouette = silhouette_score(lsa_result, dbscan_labels)
else:
    dbscan_silhouette = None
results_revised['dbscan_lsa'] = {
    'labels': dbscan_labels,
    'silhouette': dbscan_silhouette
}

# Spectral Clustering
# spectral_lsa = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', random_state=42)
# spectral_labels = spectral_lsa.fit_predict(lsa_result)
# results_revised['spectral_lsa'] = {
#     'labels': spectral_labels,
#     'silhouette': silhouette_score(lsa_result, spectral_labels)
# }

# Create summary table
evaluation_summary = pd.DataFrame({
    "Clustering Method": ["MiniBatch KMeans", "DBSCAN"],
    "Silhouette Score (LSA)": [
        results_revised['kmeans_lsa']['silhouette'],
        results_revised['dbscan_lsa']['silhouette'],
        # results_revised['spectral_lsa']['silhouette']
    ]
})

# import ace_tools as tools; tools.display_dataframe_to_user(name="Clustering Evaluation Summary", dataframe=evaluation_summary)
evaluation_summary

,Clustering Method,Silhouette Score (LSA)
0,MiniBatch KMeans,0.210066
1,DBSCAN,NaN
